For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

$$\begin{aligned}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
\end{aligned}$$

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\text{IRR}(K)} \\
      h'(K) &= \frac{\text{IRR}(K)g'(K) - g(K)\text{IRR}'(K)}{\text{IRR}(K)^2} \\
      h''(K) &= \frac{\text{IRR}(K)g''(K)-\text{IRR}''(K)g(K) -2\cdot\text{IRR}'(K)g'(K)}{\text{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\text{IRR}'(K)^2g(K)}{\text{IRR}(K)^3}.
    \end{split}
  \end{equation*}
  
For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

Let $m$ denote the payment frequency ($m=2$ for semi-annual payment frequency), and let $N = T_N-T_n$ denote the tenor of the swap (number of years), the partial derivatives on the IRR function $\text{IRR}(S)$ given by:
\begin{equation*}
\begin{split}
\text{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\text{IRR}'(K)&=-\frac{1}{K}\text{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\text{IRR}''(K)&=-\frac{2}{K}\text{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

These results will need to be generalised to handle the case for $m=2$ to be consistent with the semi-annual payment frequency swap market data provided.

---
For CMS rate payment, since $g(F)=F$, we have the derivatives:

\begin{equation*}
\begin{split}
g(K) &= K \\
g'(K) &= 1 \\
g''(K) &= 0
\end{split}
\end{equation*}

# Imports and data wrangling

OIS is in semiannual df. need to also include for quarterly period

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq
from scipy.integrate import quad
from scipy.stats import norm
from scipy.interpolate import interp1d

ois = pd.read_csv('./data/ois.csv')     # load discounting curve
fsr = pd.read_csv('./data/sabr_params.csv')     # load discounting curve
# get discount factors up to quarterly periods
# new_expiry = np.arange(0.25, ois.years.max()+.25, .25)
# ois = ois.set_index("years").reindex(new_expiry)
# ois.f = ois.f.bfill()
# ois.reset_index(inplace=True)
# ois["df"] = 1 / (1 + ois["f"] / 360) ** (90)  # get quarterly DF
# ois["df"] = ois["df"].cumprod()               # cum prod to get all DF

In [63]:
new_expiry = np.arange(0.25, ois["years"].max() + 0.25, 0.25)

# Expand the dataframe without affecting other columns
ois = ois.set_index("years").reindex(new_expiry)

# Fill forward/backward only selected columns
ois["f"] = ois["f"].bfill()  # Fill missing forward rates
ois = ois.ffill()  # Fill other missing values from above

# Reset index
ois.reset_index(inplace=True)
ois.rename(columns={"index": "years"}, inplace=True)

# Compute quarterly discount factors
ois["df"] = 1 / (1 + ois["f"] / 360) ** 90
ois["df_cumprod"] = ois["df"].cumprod()
ois.head(40)

,years,Rate,df,Tenor,Product,f,cumsum_df,df_cumprod
0,0.25,NaN,0.999376,NaN,NaN,0.002498,NaN,0.999376
1,0.50,0.00250,0.999376,6m,OIS,0.002498,0.998752,0.998752
2,0.75,0.00250,0.999127,6m,OIS,0.003493,0.998752,0.997880
3,1.00,0.00300,0.999127,1y,OIS,0.003493,1.995761,0.997009
4,1.25,0.00300,0.999127,1y,OIS,0.003495,1.995761,0.996138
5,1.50,0.00300,0.999127,1y,OIS,0.003495,2.991030,0.995268
6,1.75,0.00300,0.999127,1y,OIS,0.003495,2.991030,0.994399
7,2.00,0.00325,0.999127,2y,OIS,0.003495,3.984561,0.993531
8,2.25,0.00325,0.999114,2y,OIS,0.003545,3.984561,0.992651
9,2.50,0.00325,0.999114,2y,OIS,0.003545,4.976334,0.991771


In [55]:
1/((1+0.002498/360)**(.5*360)*(1+0.003493/360)**(.5*360)*(1+0.003495/360)**(1*360)*(1+0.003545/360)**(1*360))

0.9900147343578931

In [2]:
## IRR funcions
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative
# Payoff functions
def g_0(K):
    return K

def g_1(K):
    return 1.0

def g_2(K):
    return 0.0

def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative
    
def black76_pay(F, K, expiry, tenor, sigma):
    """
    Return value of payer swaption via Black Normal model only
    parameters
        F: forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * (sigma**2) * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = F * norm.cdf(d_1) - K * norm.cdf(d_2)
    return black_option
    
def black76_rec(F, K, expiry, tenor, sigma):
    """
    Return value of receiver swaption via Black Normal model Only
    parameters
        F: Forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * (sigma**2) * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = K * norm.cdf(-d_2) - F * norm.cdf(-d_1)
    return black_option

def SABR(F, K, T, alpha, rho, nu, beta=0.9):
    """
    Use SABR model to return sigma, set beta to be 0.9
    """
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [13]:
def rec_integrand(F, K, expiry, tenor, alpha, rho, nu, m):
    """
    Return value of receiver integrand per CMS rate payoff. 
    formula: 2nd derivative h * V_rec(per SABR valuation) 
    parameters:
        m: payment freq per year, e.g. 2 is semi-annual
        F: par swap rate
        K: strike of CMS
        expiry: years to expiry
        alpha: SABR param
        rho: SABR param
        nu: SABR param
    """
    sigma_sabr = SABR(F, K, expiry, alpha, rho, nu, beta=0.9)
    sabr_pricing = black76_rec(F, K, expiry, tenor, sigma_sabr)
    return h_2(K, m, tenor) * sabr_pricing

def pay_integrand(F, K, expiry, tenor, alpha, rho, nu, m):
    """
    Return value of receiver integrand per CMS rate payoff. 
    formula: 2nd derivative h * V_rec(per SABR valuation) 
    parameters:
        m: payment freq per year, e.g. 2 is semi-annual
        F: par swap rate
        K: strike of CMS
        expiry: years to expiry
        alpha: SABR param
        rho: SABR param
        nu: SABR param
    """
    sigma_sabr = SABR(F, K, expiry, alpha, rho, nu, beta=0.9)
    sabr_pricing = black76_pay(F, K, expiry, tenor, sigma_sabr)
    return h_2(K, m, tenor) * sabr_pricing

def v_0(F, expiry, tenor, alpha, rho, nu, m):
    """
    Return CMS pay off based of static replication discounted to today's price. Uses SABR model
    
    """
    df_expiry = ois.loc[ois.years == expiry]['df'].values[0]
    int_1 = IRR_0(F, m, tenor) * quad(lambda x: rec_integrand(F, x, expiry, tenor, alpha, rho, nu, m), 0.0, F)[0]
    int_2 = IRR_0(F, m, tenor) * quad(lambda x: pay_integrand(F, x, expiry, tenor, alpha, rho, nu, m), F, 0.1)[0]

    return df_expiry * F + int_1 + int_2



### Data prep for CMS 10y

In [14]:


tenor = 10
tenor10_forward = fsr.loc[fsr.Tenor == tenor]
tenor10_forward = tenor10_forward.copy()  # Make an explicit copy

new_expiry = np.arange(0, 5.5, 0.5)

df_10y_cms = pd.DataFrame({"Expiry": new_expiry, 
                                        "Tenor" : 10})

# interpolate sabr params
for col in ["alpha", "rho", "nu"]:
    interp_func = interp1d(tenor10_forward["Expiry"], 
                        tenor10_forward[col], 
                        kind="linear", 
                        fill_value="extrapolate")
    df_10y_cms[col] = interp_func(new_expiry)
# add par IRS  then interpolate par swap rates
tenor10_forward.loc[0] = {"Expiry": 0, "f_irs": 0.037} # from excel sheet
tenor10_forward.sort_values(by="Expiry", inplace=True)
interp_func = interp1d(tenor10_forward["Expiry"], 
                    tenor10_forward['f_irs'], 
                    kind="linear", 
                    fill_value="extrapolate")
df_10y_cms['f_irs'] = interp_func(new_expiry)

df_10y_cms.head(2)

,Expiry,Tenor,alpha,rho,nu,f_irs
0,0.0,10,0.169848,0.225443,-0.847408,0.037000
1,0.5,10,0.170400,0.244767,-0.813154,0.037703


### Calculate PV of CMS 10y for 5 years

In [15]:
ls_v0 = []
for expiry in np.arange(0.0, 5, 0.5):
    if expiry == 0:
        coupon = df_10y_cms.iloc[0]['f_irs']
        ls_v0.append(coupon)       # first payment is fixed at par swap for first period
        continue

    F = df_10y_cms.loc[df_10y_cms.Expiry == expiry].f_irs.values[0]
    alpha = df_10y_cms.loc[df_10y_cms.Expiry == expiry].alpha.values[0]
    rho = df_10y_cms.loc[df_10y_cms.Expiry == expiry].rho.values[0]
    nu = df_10y_cms.loc[df_10y_cms.Expiry == expiry].nu.values[0]
    m = 2 # semi-annual

    coupon = 1/m * v_0(F, expiry, tenor, alpha, rho, nu, m)
    print(f"{expiry}: {coupon}")
    ls_v0.append(coupon)
f"PV of a leg receiving CMS10y semi-annually over the next 5 years {np.array(ls_v0).sum():.5f}"

0.5: 0.018945364972012914
1.0: 0.019424176535304444
1.5: 0.019904643121833144
2.0: 0.020395907697261732
2.5: 0.02088296353635525
3.0: 0.02135826651885715
3.5: 0.021814056046916018
4.0: 0.02225346478295152
4.5: 0.022676816536959275


'PV of a leg receiving CMS10y semi-annually over the next 5 years 0.22466'

## Create function for forward CMS

In [33]:



tenor = 10
tenor10_forward = fsr.loc[fsr.Tenor == tenor]
tenor10_forward = tenor10_forward.copy()  # Make an explicit copy

new_expiry = np.arange(0, 5.5, 0.5)

df_10y_cms = pd.DataFrame({"Expiry": new_expiry, 
                                        "Tenor" : 10})

# interpolate sabr params
for col in ["alpha", "rho", "nu"]:
    interp_func = interp1d(tenor10_forward["Expiry"], 
                        tenor10_forward[col], 
                        kind="linear", 
                        fill_value="extrapolate")
    df_10y_cms[col] = interp_func(new_expiry)
# add par IRS  then interpolate par swap rates
tenor10_forward.loc[0] = {"Expiry": 0, "f_irs": 0.037} # from excel sheet
tenor10_forward.sort_values(by="Expiry", inplace=True)
interp_func = interp1d(tenor10_forward["Expiry"], 
                    tenor10_forward['f_irs'], 
                    kind="linear", 
                    fill_value="extrapolate")
df_10y_cms['f_irs'] = interp_func(new_expiry)

df_10y_cms.head(2)

,Expiry,Tenor,alpha,rho,nu,f_irs
0,0.0,10,0.169848,0.225443,-0.847408,0.037000
1,0.5,10,0.170400,0.244767,-0.813154,0.037703


In [45]:
ois.tail(15)

,years,Rate,df,Tenor,Product,f,cumsum_df
105,26.50,NaN,0.867669,NaN,NaN,0.005963,51.438845
106,26.75,NaN,0.866376,NaN,NaN,0.005963,NaN
107,27.00,NaN,0.865086,NaN,NaN,0.005963,52.395156
108,27.25,NaN,0.863797,NaN,NaN,0.005963,NaN
109,27.50,NaN,0.862510,NaN,NaN,0.005963,53.351196
110,27.75,NaN,0.861225,NaN,NaN,0.005963,NaN
111,28.00,NaN,0.859942,NaN,NaN,0.005963,54.306964
112,28.25,NaN,0.858661,NaN,NaN,0.005963,NaN
113,28.50,NaN,0.857382,NaN,NaN,0.005963,55.262460
114,28.75,NaN,0.856105,NaN,NaN,0.005963,NaN


In [37]:
def forward_cms_coupon(maturity, tenor, m):
    df_fwd_irs = fsr.loc[fsr.Tenor == maturity].copy()
    new_expiry = np.arange(0, tenor, 1/m)
    ls_v0 = []
    df_tmp = pd.DataFrame({"Expiry": new_expiry, 
                                        "Tenor" : 10})
    
    # interpolate sabr params
    for col in ["alpha", "rho", "nu"]:
        interp_func = interp1d(df_fwd_irs["Expiry"], 
                            df_fwd_irs[col], 
                            kind="linear", 
                            fill_value="extrapolate")
        df_tmp[col] = interp_func(new_expiry)
    # add par IRS  then interpolate par swap rates
    df_fwd_irs.loc[0] = {"Expiry": 0, "f_irs": 0.037} # TODO from excel sheet
    df_fwd_irs.sort_values(by="Expiry", inplace=True)
    interp_func = interp1d(df_fwd_irs["Expiry"], 
                        df_fwd_irs['f_irs'], 
                        kind="linear", 
                        fill_value="extrapolate")
    df_tmp['f_irs'] = interp_func(new_expiry)

    for expiry in new_expiry:
        if expiry == 0:
            coupon = 1/m * df_tmp.iloc[0]['f_irs']
            print(f"{expiry}: {coupon}")
            ls_v0.append(coupon)       # first payment is fixed at par swap for first period
            continue

        F = df_tmp.loc[df_tmp.Expiry == expiry].f_irs.values[0]
        alpha = df_tmp.loc[df_tmp.Expiry == expiry].alpha.values[0]
        rho = df_tmp.loc[df_tmp.Expiry == expiry].rho.values[0]
        nu = df_tmp.loc[df_tmp.Expiry == expiry].nu.values[0]

        coupon = 1/m * v_0(F, expiry, tenor, alpha, rho, nu, m)
        print(f"{expiry}: {coupon}")
        ls_v0.append(coupon)

    return np.array(ls_v0).sum()

forward_cms_coupon(10, 5, 2)

0.0: 0.0185
0.5: 0.018889941097890366
1.0: 0.01929222731915649
1.5: 0.019680628369221468
2.0: 0.020074055217903078
2.5: 0.020464633137462756
3.0: 0.0208487529286184
3.5: 0.021220418189455276
4.0: 0.021583165341112695
4.5: 0.021936873921735912


0.20249069552255647

In [34]:

for expiry in np.arange(0.0, 5, 0.5):
    if expiry == 0:
        coupon = df_10y_cms.iloc[0]['f_irs']
        ls_v0.append(coupon)       # first payment is fixed at par swap for first period
        continue

    F = df_10y_cms.loc[df_10y_cms.Expiry == expiry].f_irs.values[0]
    alpha = df_10y_cms.loc[df_10y_cms.Expiry == expiry].alpha.values[0]
    rho = df_10y_cms.loc[df_10y_cms.Expiry == expiry].rho.values[0]
    nu = df_10y_cms.loc[df_10y_cms.Expiry == expiry].nu.values[0]
    m = 2 # semi-annual

    coupon = 1/m * v_0(F, expiry, tenor, alpha, rho, nu, m)
    print(f"{expiry}: {coupon}")
    ls_v0.append(coupon)
f"PV of a leg receiving CMS10y semi-annually over the next 5 years {np.array(ls_v0).sum():.5f}"

0.5: 0.018945364972012914
1.0: 0.019424176535304444
1.5: 0.019904643121833144
2.0: 0.020395907697261732
2.5: 0.02088296353635525
3.0: 0.02135826651885715
3.5: 0.021814056046916018
4.0: 0.02225346478295152
4.5: 0.022676816536959275


'PV of a leg receiving CMS10y semi-annually over the next 5 years 0.63488'

### Data prep for CMS 2y

In [16]:
tenor = 2
tenor2_forward = fsr.loc[fsr.Tenor == tenor]
tenor2_forward = tenor2_forward.copy()  # Make an explicit copy

new_expiry = np.arange(0, 10, 0.25)

df_2y_cms = pd.DataFrame({"Expiry": new_expiry, 
                           

                            "Tenor" : 2})

# interpolate sabr params
for col in ["alpha", "rho", "nu"]:
    interp_func = interp1d(tenor2_forward["Expiry"], 
                        tenor2_forward[col], 
                        kind="linear", 
                        fill_value="extrapolate")
    df_2y_cms[col] = interp_func(new_expiry)
# add par IRS  then interpolate par swap rates
tenor2_forward.loc[0] = {"Expiry": 0, "f_irs": 0.03}  # from excel sheet
tenor2_forward.sort_values(by="Expiry", inplace=True)
interp_func = interp1d(tenor2_forward["Expiry"], 
                    tenor2_forward['f_irs'], 
                    kind="linear", 
                    fill_value="extrapolate")
df_2y_cms['f_irs'] = interp_func(new_expiry)

df_2y_cms.head(2)

,Expiry,Tenor,alpha,rho,nu,f_irs
0,0.00,2,0.180946,0.519203,-1.828011,0.030000
1,0.25,2,0.181876,0.520592,-1.789689,0.030804


### Calculate PV of CMS 2y for 10 years

In [18]:
ls_v0 = []

for expiry in np.arange(0, 10, 0.25):
    if expiry == 0:
        coupon = df_2y_cms.iloc[0]['f_irs']
        ls_v0.append(coupon)       # first payment is fixed at par swap for first period
        print(f"{expiry}: {coupon}")
        continue

    F = df_2y_cms.loc[df_2y_cms.Expiry == expiry].f_irs.values[0]
    alpha = df_2y_cms.loc[df_2y_cms.Expiry == expiry].alpha.values[0]
    rho = df_2y_cms.loc[df_2y_cms.Expiry == expiry].rho.values[0]
    nu = df_2y_cms.loc[df_2y_cms.Expiry == expiry].nu.values[0]
    m = 4 # quarterly

    coupon = 1/m * v_0(F, expiry, tenor, alpha, rho, nu, m)
    ls_v0.append(coupon)
    print(f"{expiry}: {coupon}")

f"PV of a leg receiving CMS10y semi-annually over the next 10 years {np.array(ls_v0).sum()}"

0.0: 0.03
0.25: 0.007702211322276684
0.5: 0.007909002630507337
0.75: 0.008118923774189938
1.0: 0.008332487934766347
1.25: 0.008453174806783882
1.5: 0.00857432166052755
1.75: 0.008695115445417899
2.0: 0.008815026456660758
2.25: 0.008933608775275086
2.5: 0.009050765113162326
2.75: 0.0091663815221661
3.0: 0.009280407200010906
3.25: 0.009391917590108904
3.5: 0.009501837427833303
3.75: 0.009610215742753033
4.0: 0.009717118564023592
4.25: 0.009822504488833626
4.5: 0.009926579311756395
4.75: 0.010029438443345808
5.0: 0.010131182943840654
5.25: 0.010168463721496336
5.5: 0.010205357877496208
5.75: 0.010241865291896435
6.0: 0.010277986887090385
6.25: 0.010313724419434437
6.5: 0.010349080313524269
6.75: 0.010384057529581395
7.0: 0.010418659456738644
7.25: 0.010451619587047196
7.5: 0.010484206592122305
7.75: 0.01051642471996132
8.0: 0.010548278386931008
8.25: 0.010579772139333152
8.5: 0.010610910621785555
8.75: 0.010641698551169821
9.0: 0.01067214069514872
9.25: 0.010702241854452335
9.5: 0.0107320

'PV of a leg receiving CMS10y semi-annually over the next 10 years 0.41022215715007104'

In [11]:
# TODO make function to get CMS for given tenor